# Ferrero Field Sales Analysis

Ferrero Field Sales manages a large field team of over 200 people, that visit top Grocery and Wholesale stores in the
UK, on a very regular basis. Their objective is to drive sales of selected chocolate confectionery SKUs, by using their relationships at a store level to gain incremental space/ ensure compliance to
nationally agreed planograms and promotional paid for spaces, identify and resolve store level stock
issues etc.

Attached is a data dump for December 2020 and 2019. We would like you to analyse the data and
prepare a short presentation that discusses the drivers and drainers of our performance 2020 vs 2019, and provide recommendations on strategy for the next year.

- For information, an “Intervention” is an action a Field Exec has taken in store, to actively influence
the sales of the SKUs we focus on. 

- Net Revenue is essentially a modelled Incremental Uplift metric.

- Qtli is a custom Volume based metric.

## Summary

### Imports

In [1]:
import sys

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import numpy as np

In [2]:
print(f'pandas:{pd.__version__}')
print(f'seaborn:{sns.__version__}')
print(f'altair:{alt.__version__}')
print(f'python:{sys.version}')

pandas:2.0.1
seaborn:0.12.2
altair:5.0.0
python:3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]


In [3]:
# Ignore warnings
pd.option_context('mode.chained_assignment', None)

In [4]:
BASE = r"C:\\Users\\anand\\Documents\\Ferrero_field_sales_analysis"

PATH = BASE + r"\\data\\"

## Read Data

In [5]:
# Read excel
df = pd.read_excel(PATH + r"December Data 1819 and 1920 (1).xlsx")

# Preview
df.head(10)

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0
5,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Replenishment,23,380.0,1.0
6,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Site SEL / POS,9,93.0,0.2
7,20182019,13,2018-12-02,2018-12-02,Grocery,Asda,Bueno,Amend Stock Level,11,464.0,1.1
8,20182019,14,2018-12-02,2018-12-03,Grocery,Asda,Bueno,Amend Stock Level,52,2667.0,5.5
9,20182019,14,2018-12-02,2018-12-04,Grocery,Asda,Bueno,Amend Stock Level,42,2254.0,4.4


In [6]:
# Shape of data
df.shape

(15990, 11)

In [7]:
# Preview columns
df.head().T

,0,1,2,3,4
FinancialYear,20182019,20182019,20182019,20182019,20182019
FYWeek,13,13,13,13,13
WeekComm,2018-11-25 00:00:00,2018-11-25 00:00:00,2018-11-25 00:00:00,2018-11-25 00:00:00,2018-11-25 00:00:00
Date,2018-12-01 00:00:00,2018-12-01 00:00:00,2018-12-01 00:00:00,2018-12-01 00:00:00,2018-12-01 00:00:00
Channel,Grocery,Grocery,Grocery,Grocery,Grocery
Group,Asda,Asda,Asda,Asda,Asda
Brand,Bueno,Bueno,Bueno,Bueno,Bueno
Top Intervention,Amend Stock Level,Local Display,Merchandising,National Display,Place Order
Count of Interventions,26,5,25,6,1
Net revenue,1586.0,241.0,424.0,428.0,13.0


In [8]:
# Mean and quantiles
df.describe().T

,count,mean,min,25%,50%,75%,max,std
FinancialYear,15990.0,20187120.823452,20182019.0,20182019.0,20192020.0,20192020.0,20192020.0,4999.629692
FYWeek,15990.0,14.983865,13.0,14.0,15.0,16.0,17.0,0.996455
WeekComm,15990,2019-06-12 20:31:14.971857408,2018-11-25 00:00:00,2018-12-09 00:00:00,2019-12-01 00:00:00,2019-12-08 00:00:00,2019-12-22 00:00:00,NaN
Date,15990,2019-06-15 20:30:42.551594752,2018-12-01 00:00:00,2018-12-12 00:00:00,2019-12-01 12:00:00,2019-12-11 00:00:00,2019-12-24 00:00:00,NaN
Count of Interventions,15990.0,32.755722,1.0,2.0,7.0,26.0,1880.0,78.630346
Net revenue,15437.0,903.009263,0.0,7.0,70.0,492.0,95389.0,3151.086009
Quintali,15437.0,1.079543,0.0,0.0,0.1,0.7,86.9,3.439377


In [9]:
# Sum
df['Net revenue'].sum()

13939754.0

In [10]:
# Quantiles
df['Net revenue'].quantile([0.1,0.5,0.75,0.80,0.85,0.90,0.95,1])

0.10        0.0
0.50       70.0
0.75      492.0
0.80      748.8
0.85     1168.2
0.90     1953.0
0.95     4132.8
1.00    95389.0
Name: Net revenue, dtype: float64

In [11]:
len(df[df['Net revenue'] > 10000])

264

In [12]:
# Sum
df['Count of Interventions'].sum()

523764

In [13]:
# Sort by net revenue
df.sort_values('Net revenue', ascending=False).head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali
6852,20182019,16,2018-12-16,2018-12-19,Grocery,Tesco,Pralines,Local Display,721,95389.0,86.9
7093,20192020,16,2019-12-15,2019-12-16,Grocery,Tesco,Pralines,Replenishment,1098,62654.0,59.0
3393,20182019,15,2018-12-09,2018-12-12,Grocery,Morrisons,Thorntons,Local Display,374,54723.0,83.6
3184,20192020,14,2019-12-01,2019-12-04,Grocery,Morrisons,Pralines,National Display,522,54722.0,49.7
7030,20192020,15,2019-12-08,2019-12-09,Grocery,Tesco,Pralines,Replenishment,945,51584.0,47.0


In [14]:
# Percentage of highest revenue
95389.0/df['Net revenue'].sum() * 100

0.6842947156743225

In [15]:
# Percentage of high revenue interventions
df[df['Net revenue'] > 10000]['Net revenue'].sum() /df['Net revenue'].sum() * 100

37.843817042969334

In [16]:
# Value count
df.Channel.value_counts()

Channel
Grocery    7805
Ireland    5587
Impulse    2598
Name: count, dtype: int64

In [17]:
# Value count
df.Group.value_counts()

Group
Tesco           3274
Asda            2633
Sainsburys      2124
Morrisons       1649
Dunnes          1333
Super Valu      1033
Bestway          521
BWG              440
Dhamecha         408
Batleys          354
Booker           314
Hendersons       286
Centra           281
Retail Clubs     220
Independent      201
Landmark         186
Symbols          179
Spar             105
Coop              97
Eurospar          78
Parfetts          72
Todays            63
Hancocks          47
Booths            29
Vivo              23
Sugro             16
Nisa Today        10
Blakemore          7
WH Smith           7
Name: count, dtype: int64

In [18]:
# Value count
df['Top Intervention'].value_counts()

Top Intervention
Merchandising                  3211
Replenishment                  2872
Local Display                  2216
Site SEL / POS                 1601
Place Order                    1484
Permanent Additional Facing     761
Sited Clipstrips                631
National Display                628
Fix Book stock error            571
Amend Stock Level               540
Adjust shelf capacity           374
No Action                       340
No Intervention                 225
Merchandised                    133
Distribution Gain               121
LVA - Merchandised               69
Site SEL/POS                     63
HVA - Local Display              43
Placed Order                     26
Site SEL/ POS                    24
HVA - Sited POS                  23
LVA - Replenishment              15
HVA - Distribution Gain          11
Sited Clip Strips                 3
Sited POS                         3
LVA - Sited SEL                   1
HVA - Placed Order                1
Name: count

## December 2019

In [19]:
# Filter by 2019
df_2019 = df[df.FinancialYear==20182019]

# Preview
df_2019.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0


In [20]:
# Shape of data
df_2019.shape

(7833, 11)

In [21]:
# Mean and quantiles
df_2019.describe().T

,count,mean,min,25%,50%,75%,max,std
FinancialYear,7833.0,20182019.0,20182019.0,20182019.0,20182019.0,20182019.0,20182019.0,0.0
FYWeek,7833.0,14.871441,13.0,14.0,15.0,16.0,17.0,1.006715
WeekComm,7833,2018-12-08 16:07:43.270777600,2018-11-25 00:00:00,2018-12-02 00:00:00,2018-12-09 00:00:00,2018-12-16 00:00:00,2018-12-23 00:00:00,NaN
Date,7833,2018-12-11 17:06:32.952891648,2018-12-01 00:00:00,2018-12-06 00:00:00,2018-12-11 00:00:00,2018-12-18 00:00:00,2018-12-24 00:00:00,NaN
Count of Interventions,7833.0,26.594025,1.0,2.0,7.0,22.0,767.0,58.851991
Net revenue,7333.0,919.339015,0.0,9.0,79.0,521.0,95389.0,3185.978527
Quintali,7333.0,1.088545,0.0,0.0,0.1,0.7,86.9,3.517266


In [22]:
# Sum
df_2019['Count of Interventions'].sum()

208311

In [23]:
# Sum
df_2019['Net revenue'].sum()

6741513.0

In [24]:
# High revenue intervention count
len(df_2019[df_2019['Net revenue'] > 10000])

130

In [25]:
# Sum
df_2019['Quintali'].sum()

7982.299999999999

In [26]:
# Value count
df_2019.Channel.value_counts()

Channel
Grocery    3906
Ireland    2555
Impulse    1372
Name: count, dtype: int64

In [27]:
# Value count
df_2019.Group.value_counts()

Group
Tesco           1517
Asda            1298
Sainsburys      1074
Morrisons        892
Dunnes           671
Super Valu       618
Bestway          291
Booker           208
Batleys          201
Dhamecha         201
Spar             105
Landmark         104
Coop              97
Independent       95
Centra            88
Retail Clubs      80
Eurospar          78
Symbols           64
Todays            40
Hancocks          36
Parfetts          29
Vivo              23
Sugro             16
Blakemore          7
Name: count, dtype: int64

In [28]:
# Value count
df_2019['Top Intervention'].value_counts()

Top Intervention
Replenishment                  1449
Merchandising                  1433
Local Display                  1057
Place Order                     840
Site SEL / POS                  771
Amend Stock Level               534
Permanent Additional Facing     503
Sited Clipstrips                345
National Display                276
No Action                       268
No Intervention                  76
LVA - Merchandised               69
Site SEL/POS                     63
Distribution Gain                55
HVA - Local Display              43
HVA - Sited POS                  23
LVA - Replenishment              15
HVA - Distribution Gain          11
LVA - Sited SEL                   1
HVA - Placed Order                1
Name: count, dtype: int64

## December 2020

In [29]:
# Filter by 2020
df_2020 = df[df.FinancialYear==20192020]

# Preview
df_2020.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali
156,20192020,14,2019-12-01,2019-12-01,Grocery,Asda,Bueno,Adjust shelf capacity,1,40.0,0.1
157,20192020,14,2019-12-01,2019-12-02,Grocery,Asda,Bueno,Adjust shelf capacity,9,387.0,1.1
158,20192020,14,2019-12-01,2019-12-03,Grocery,Asda,Bueno,Adjust shelf capacity,9,286.0,0.6
159,20192020,14,2019-12-01,2019-12-04,Grocery,Asda,Bueno,Adjust shelf capacity,7,273.0,1.0
160,20192020,14,2019-12-01,2019-12-05,Grocery,Asda,Bueno,Adjust shelf capacity,2,48.0,0.1


In [30]:
# Shape of data
df_2020.shape

(8157, 11)

In [31]:
# Mean and quantiles
df_2020.describe().T

,count,mean,min,25%,50%,75%,max,std
FinancialYear,8157.0,20192020.0,20192020.0,20192020.0,20192020.0,20192020.0,20192020.0,0.0
FYWeek,8157.0,15.091823,14.0,14.0,15.0,16.0,17.0,0.974429
WeekComm,8157,2019-12-08 15:25:34.534755328,2019-12-01 00:00:00,2019-12-01 00:00:00,2019-12-08 00:00:00,2019-12-15 00:00:00,2019-12-22 00:00:00,NaN
Date,8157,2019-12-11 14:28:01.500551680,2019-12-01 00:00:00,2019-12-06 00:00:00,2019-12-11 00:00:00,2019-12-17 00:00:00,2019-12-24 00:00:00,NaN
Count of Interventions,8157.0,38.672674,1.0,2.0,8.0,31.0,1880.0,93.397891
Net revenue,8104.0,888.233095,0.0,6.0,64.0,465.5,62654.0,3119.299889
Quintali,8104.0,1.071397,0.0,0.0,0.1,0.6,59.4,3.367542


In [32]:
# Sum
df_2020['Count of Interventions'].sum()

315453

In [33]:
# Sum
df_2020['Net revenue'].sum()

7198241.0

In [34]:
# Sum
df_2020['Quintali'].sum()

8682.6

In [35]:
# Value count
df_2020.Channel.value_counts()

Channel
Grocery    3899
Ireland    3032
Impulse    1226
Name: count, dtype: int64

In [36]:
# Value count
df_2020.Group.value_counts()

Group
Tesco           1757
Asda            1335
Sainsburys      1050
Morrisons        757
Dunnes           662
BWG              440
Super Valu       415
Hendersons       286
Bestway          230
Dhamecha         207
Centra           193
Batleys          153
Retail Clubs     140
Symbols          115
Booker           106
Independent      106
Landmark          82
Parfetts          43
Booths            29
Todays            23
Hancocks          11
Nisa Today        10
WH Smith           7
Name: count, dtype: int64

In [37]:
# Value count
df_2020['Top Intervention'].value_counts()

Top Intervention
Merchandising                  1778
Replenishment                  1423
Local Display                  1159
Site SEL / POS                  830
Place Order                     644
Fix Book stock error            571
Adjust shelf capacity           374
National Display                352
Sited Clipstrips                286
Permanent Additional Facing     258
No Intervention                 149
Merchandised                    133
No Action                        72
Distribution Gain                66
Placed Order                     26
Site SEL/ POS                    24
Amend Stock Level                 6
Sited Clip Strips                 3
Sited POS                         3
Name: count, dtype: int64

## Financial Year Week

In [38]:
# Preview
df_2019.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0


In [39]:
# Select columns
cols = [
    'FYWeek',
    'Count of Interventions',
    'Net revenue',
    'Quintali',
]

In [40]:
# Group by column
fyw_2019 = df_2019[cols].groupby(
    'FYWeek',
    
).mean().reset_index()

# Preview
fyw_2019.head()

,FYWeek,Count of Interventions,Net revenue,Quintali
0,13,14.788168,496.936543,0.583807
1,14,23.641971,892.510833,1.019667
2,15,30.658688,1059.871653,1.266944
3,16,28.382184,953.702381,1.146528
4,17,30.294118,523.948980,0.723469


In [41]:
# Calculating profit index
fyw_2019['Profit index'] = fyw_2019['Net revenue'] / fyw_2019['Count of Interventions']

# Preview
fyw_2019.head()

,FYWeek,Count of Interventions,Net revenue,Quintali,Profit index
0,13,14.788168,496.936543,0.583807,33.603658
1,14,23.641971,892.510833,1.019667,37.751118
2,15,30.658688,1059.871653,1.266944,34.570026
3,16,28.382184,953.702381,1.146528,33.602149
4,17,30.294118,523.948980,0.723469,17.295403


In [42]:
# Group by column
fyw_2020 = df_2020[cols].groupby(
    'FYWeek',
    
).mean().reset_index()

# Preview
fyw_2020.head()

,FYWeek,Count of Interventions,Net revenue,Quintali
0,14,33.819597,872.129274,1.043910
1,15,39.666124,1053.603596,1.251369
2,16,41.826408,832.636194,1.003965
3,17,44.978723,547.463830,0.761277


In [43]:
# Calculating profit index
fyw_2020['Profit index'] = fyw_2020['Net revenue'] / fyw_2020['Count of Interventions']

# Preview
fyw_2020.head()

,FYWeek,Count of Interventions,Net revenue,Quintali,Profit index
0,14,33.819597,872.129274,1.043910,25.787690
1,15,39.666124,1053.603596,1.251369,26.561798
2,16,41.826408,832.636194,1.003965,19.906949
3,17,44.978723,547.463830,0.761277,12.171618


In [44]:
# Merge
com = fyw_2019.merge(fyw_2020,on='FYWeek')

# Preview
com.head()

,FYWeek,Count of Interventions_x,Net revenue_x,Quintali_x,Profit index_x,Count of Interventions_y,Net revenue_y,Quintali_y,Profit index_y
0,14,23.641971,892.510833,1.019667,37.751118,33.819597,872.129274,1.043910,25.787690
1,15,30.658688,1059.871653,1.266944,34.570026,39.666124,1053.603596,1.251369,26.561798
2,16,28.382184,953.702381,1.146528,33.602149,41.826408,832.636194,1.003965,19.906949
3,17,30.294118,523.948980,0.723469,17.295403,44.978723,547.463830,0.761277,12.171618


In [45]:
# Rename columns
com.columns = [
    'FYWeek',
    '2019',
    '2019 ',
    '2019  ',
    '2019   ',
    '2020',
    '2020 ',
    '2020  ',
    '2020   '
]

com.head()

,FYWeek,2019,2019,2019,2019,2020,2020,2020,2020
0,14,23.641971,892.510833,1.019667,37.751118,33.819597,872.129274,1.043910,25.787690
1,15,30.658688,1059.871653,1.266944,34.570026,39.666124,1053.603596,1.251369,26.561798
2,16,28.382184,953.702381,1.146528,33.602149,41.826408,832.636194,1.003965,19.906949
3,17,30.294118,523.948980,0.723469,17.295403,44.978723,547.463830,0.761277,12.171618


In [46]:
chart = alt.Chart(com).transform_fold(
    ['2019   ', '2020   '],
).mark_line(point=True).encode(
    x=alt.X('FYWeek:O',title = 'Financial Year Week'),
    y=alt.Y('value:Q', title='Average Profit Index'),
    color='key:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average Profit Index per Financial Year Week ',
    height=300,
    width=600
)


rule = alt.Chart(com).transform_fold(
    ['2019   '],
).mark_rule(color='green').encode(
    y='mean(value):Q')

rule2 = alt.Chart(com).transform_fold(
    ['2020   '],
).mark_rule(color='purple').encode(
    y='mean(value):Q')

chart + rule + rule2

alt.LayerChart(...)

In [47]:
alt.Chart(com).transform_fold(
    ['2019', '2020'],
).mark_line(point=True).encode(
    x=alt.X('FYWeek:O',title = 'Financial Year Week'),
    y=alt.Y('value:Q', title='Average Number of Interventions'),
    color='key:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average Number of Interventions per Financial Year Week ',
    height=300,
    width=600
)

alt.Chart(...)

In [48]:
alt.Chart(com).transform_fold(
    ['2019 ', '2020 '],
).mark_line(point=True).encode(
    x=alt.X('FYWeek:O',title = 'Financial Year Week'),
    y=alt.Y('value:Q', title='Average revenue'),
    color='key:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average revenue per Financial Year Week ',
    height=300,
    width=600
)

alt.Chart(...)

## Date

In [49]:
# Preview
df_2019.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0


In [50]:
# Get day from date
df_2019['Day'] = df_2019.Date.dt.day

df_2020['Day'] = df_2020.Date.dt.day

# Preview
df_2020.head()

C:\Users\anand\AppData\Local\Temp\ipykernel_11468\3100181779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['Day'] = df_2019.Date.dt.day
C:\Users\anand\AppData\Local\Temp\ipykernel_11468\3100181779.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['Day'] = df_2020.Date.dt.day


,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali,Day
156,20192020,14,2019-12-01,2019-12-01,Grocery,Asda,Bueno,Adjust shelf capacity,1,40.0,0.1,1
157,20192020,14,2019-12-01,2019-12-02,Grocery,Asda,Bueno,Adjust shelf capacity,9,387.0,1.1,2
158,20192020,14,2019-12-01,2019-12-03,Grocery,Asda,Bueno,Adjust shelf capacity,9,286.0,0.6,3
159,20192020,14,2019-12-01,2019-12-04,Grocery,Asda,Bueno,Adjust shelf capacity,7,273.0,1.0,4
160,20192020,14,2019-12-01,2019-12-05,Grocery,Asda,Bueno,Adjust shelf capacity,2,48.0,0.1,5


In [51]:
# Select columns
cols = [
    'Day',
    'Count of Interventions',
    'Net revenue',
    'Quintali',
]

In [52]:
# Group by column
fyw_2019 = df_2019[cols].groupby(
    'Day',
    
).mean().reset_index()

# Preview
fyw_2019.head()

,Day,Count of Interventions,Net revenue,Quintali
0,1,18.857143,638.840580,0.763406
1,2,8.301980,280.552486,0.309945
2,3,24.309211,1080.982005,1.226221
3,4,25.643204,1020.988338,1.111370
4,5,25.295154,1016.691906,1.155875


In [53]:
# Calculating profit index
fyw_2019['Profit index'] = fyw_2019['Net revenue'] / fyw_2019['Count of Interventions']

# Preview
fyw_2019.head()

,Day,Count of Interventions,Net revenue,Quintali,Profit index
0,1,18.857143,638.840580,0.763406,33.877910
1,2,8.301980,280.552486,0.309945,33.793442
2,3,24.309211,1080.982005,1.226221,44.468001
3,4,25.643204,1020.988338,1.111370,39.815163
4,5,25.295154,1016.691906,1.155875,40.193149


In [54]:
# Average profit index
fyw_2019['Profit index'].mean()

33.51909518424006

In [55]:
# Group by column
fyw_2020 = df_2020[cols].groupby(
    'Day',
    
).mean().reset_index()

# Preview
fyw_2020.head()

,Day,Count of Interventions,Net revenue,Quintali
0,1,15.395062,453.092593,0.583333
1,2,31.322519,857.161850,1.110597
2,3,38.848018,915.549889,1.053880
3,4,35.083691,933.405579,1.078326
4,5,37.011962,898.377990,1.036364


In [56]:
# Calculating profit index
fyw_2020['Profit index'] = fyw_2020['Net revenue'] / fyw_2020['Count of Interventions']

# Preview
fyw_2020.head()

,Day,Count of Interventions,Net revenue,Quintali,Profit index
0,1,15.395062,453.092593,0.583333,29.431034
1,2,31.322519,857.161850,1.110597,27.365674
2,3,38.848018,915.549889,1.053880,23.567480
3,4,35.083691,933.405579,1.078326,26.605113
4,5,37.011962,898.377990,1.036364,24.272639


In [57]:
# Average profit index
fyw_2020['Profit index'].mean()

22.813162015457937

In [58]:
# Merge
com = fyw_2019.merge(fyw_2020,on='Day')

# Preview
com.head()

,Day,Count of Interventions_x,Net revenue_x,Quintali_x,Profit index_x,Count of Interventions_y,Net revenue_y,Quintali_y,Profit index_y
0,1,18.857143,638.840580,0.763406,33.877910,15.395062,453.092593,0.583333,29.431034
1,2,8.301980,280.552486,0.309945,33.793442,31.322519,857.161850,1.110597,27.365674
2,3,24.309211,1080.982005,1.226221,44.468001,38.848018,915.549889,1.053880,23.567480
3,4,25.643204,1020.988338,1.111370,39.815163,35.083691,933.405579,1.078326,26.605113
4,5,25.295154,1016.691906,1.155875,40.193149,37.011962,898.377990,1.036364,24.272639


In [59]:
# Rename columns
com.columns = [
    'Day',
    '2019',
    '2019 ',
    '2019  ',
    '2019   ',
    '2020',
    '2020 ',
    '2020  ',
    '2020   '
]

com.head()

,Day,2019,2019,2019,2019,2020,2020,2020,2020
0,1,18.857143,638.840580,0.763406,33.877910,15.395062,453.092593,0.583333,29.431034
1,2,8.301980,280.552486,0.309945,33.793442,31.322519,857.161850,1.110597,27.365674
2,3,24.309211,1080.982005,1.226221,44.468001,38.848018,915.549889,1.053880,23.567480
3,4,25.643204,1020.988338,1.111370,39.815163,35.083691,933.405579,1.078326,26.605113
4,5,25.295154,1016.691906,1.155875,40.193149,37.011962,898.377990,1.036364,24.272639


In [60]:
chart = alt.Chart(com).transform_fold(
    ['2019   ', '2020   '],
).mark_line().encode(
    x=alt.X('Day:O',title = 'Day'),
    y=alt.Y('value:Q', title='Average Profit Index'),
    color='key:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average Profit Index per Day ',
    height=300,
    width=600
)

rule = alt.Chart(com).transform_fold(
    ['2019   '],
).mark_rule(color='green').encode(
    y='mean(value):Q')

rule2 = alt.Chart(com).transform_fold(
    ['2020   '],
).mark_rule(color='purple').encode(
    y='mean(value):Q')

chart + rule + rule2

alt.LayerChart(...)

In [61]:
alt.Chart(com).transform_fold(
    ['2019', '2020'],
).mark_line().encode(
    x=alt.X('Day:O',title = 'Day'),
    y=alt.Y('value:Q', title='Average Number of Interventions'),
    color='key:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average Number of Interventions per Day ',
    height=300,
    width=600
)

alt.Chart(...)

In [62]:
alt.Chart(com).transform_fold(
    ['2019 ', '2020 '],
).mark_line().encode(
    x=alt.X('Day:O',title = 'Day'),
    y=alt.Y('value:Q', title='Average revenue'),
    color='key:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average revenue per Day ',
    height=300,
    width=600
)

alt.Chart(...)

In [63]:
df_2019[df_2019.Day == 22].sort_values('Count of Interventions', ascending = False).head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali,Day
6855,20182019,16,2018-12-16,2018-12-22,Grocery,Tesco,Pralines,Local Display,239,10749.0,10.6,22
6891,20182019,16,2018-12-16,2018-12-22,Grocery,Tesco,Pralines,Replenishment,211,7314.0,7.5,22
7299,20182019,16,2018-12-16,2018-12-22,Grocery,Tesco,Thorntons,Replenishment,211,4156.0,4.7,22
1238,20182019,16,2018-12-16,2018-12-22,Grocery,Asda,Pralines,Replenishment,182,6821.0,6.0,22
1651,20182019,16,2018-12-16,2018-12-22,Grocery,Asda,Thorntons,Replenishment,182,4009.0,4.3,22


In [64]:
df_2019[df_2019.Day == 22].sort_values('Count of Interventions', ascending = False).head(20).Brand.value_counts()

Brand
Thorntons    9
Pralines     7
Kinder       3
Nutella      1
Name: count, dtype: int64

In [65]:
df_2019[df_2019.Day == 22].sort_values('Count of Interventions', ascending = False).head(20).Group.value_counts()

Group
Tesco         9
Asda          7
Sainsburys    3
Morrisons     1
Name: count, dtype: int64

In [66]:
df_2019[df_2019.Day == 23].sort_values('Count of Interventions', ascending = False).head(20).Channel.value_counts()

Channel
Grocery    20
Name: count, dtype: int64

In [67]:
df_2020[df_2020.Day == 23].sort_values('Count of Interventions', ascending = False).head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali,Day
7129,20192020,17,2019-12-22,2019-12-23,Grocery,Tesco,Pralines,Replenishment,914,17561.0,17.5,23
7522,20192020,17,2019-12-22,2019-12-23,Grocery,Tesco,Thorntons,Replenishment,847,6864.0,7.1,23
1463,20192020,17,2019-12-22,2019-12-23,Grocery,Asda,Pralines,Replenishment,738,17867.0,19.2,23
1856,20192020,17,2019-12-22,2019-12-23,Grocery,Asda,Thorntons,Replenishment,635,4190.0,5.2,23
6695,20192020,17,2019-12-22,2019-12-23,Grocery,Tesco,Nutella,Merchandising,458,2873.0,8.7,23


In [68]:
df_2020[df_2020.Day == 23].sort_values('Count of Interventions', ascending = False).head(20).Brand.value_counts()

Brand
Thorntons    6
Pralines     5
Nutella      4
Kinder       4
Bueno        1
Name: count, dtype: int64

In [69]:
df_2019[df_2019.Day == 23].sort_values('Count of Interventions', ascending = False).head(20).Group.value_counts()

Group
Asda          8
Tesco         7
Morrisons     3
Sainsburys    2
Name: count, dtype: int64

In [70]:
df_2019[df_2019.Day == 23].sort_values('Count of Interventions', ascending = False).head(20).Channel.value_counts()

Channel
Grocery    20
Name: count, dtype: int64

## Channel

In [71]:
# Preview
df_2019.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali,Day
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9,1
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4,1
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7,1
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0,1
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0,1


In [72]:
# Unique values
df_2019.Channel.value_counts()

Channel
Grocery    3906
Ireland    2555
Impulse    1372
Name: count, dtype: int64

In [73]:
# Select columns
cols = [
    'Channel',
    'Count of Interventions',
    'Net revenue',
    'Quintali',
]

In [74]:
# Group by column
fyw_2019 = df_2019[cols].groupby(
    'Channel',
    
).mean().reset_index()

# Preview
fyw_2019.head()

,Channel,Count of Interventions,Net revenue,Quintali
0,Grocery,42.178443,1504.359445,1.713361
1,Impulse,6.365889,235.804557,0.378089
2,Ireland,13.631311,307.503158,0.425684


In [75]:
# Calculating profit index
fyw_2019['Profit index'] = fyw_2019['Net revenue'] / fyw_2019['Count of Interventions']

# Preview
fyw_2019.head()

,Channel,Count of Interventions,Net revenue,Quintali,Profit index
0,Grocery,42.178443,1504.359445,1.713361,35.666547
1,Impulse,6.365889,235.804557,0.378089,37.041888
2,Ireland,13.631311,307.503158,0.425684,22.558590


In [76]:
fyw_2019['Profit index'].mean()

31.755675152707315

In [77]:
# Group by column
fyw_2020 = df_2020[cols].groupby(
    'Channel',
    
).mean().reset_index()

# Preview
fyw_2020.head()

,Channel,Count of Interventions,Net revenue,Quintali
0,Grocery,63.412926,1535.012311,1.744883
1,Impulse,12.851550,284.794894,0.477106
2,Ireland,17.298813,289.965017,0.435215


In [78]:
# Calculating profit index
fyw_2020['Profit index'] = fyw_2020['Net revenue'] / fyw_2020['Count of Interventions']

# Preview
fyw_2020.head()

,Channel,Count of Interventions,Net revenue,Quintali,Profit index
0,Grocery,63.412926,1535.012311,1.744883,24.206615
1,Impulse,12.851550,284.794894,0.477106,22.160354
2,Ireland,17.298813,289.965017,0.435215,16.762134


In [79]:
fyw_2020['Profit index'].mean()

21.04303446797807

In [80]:
# Merge
com = fyw_2019.merge(fyw_2020,on='Channel')

# Preview
com.head()

,Channel,Count of Interventions_x,Net revenue_x,Quintali_x,Profit index_x,Count of Interventions_y,Net revenue_y,Quintali_y,Profit index_y
0,Grocery,42.178443,1504.359445,1.713361,35.666547,63.412926,1535.012311,1.744883,24.206615
1,Impulse,6.365889,235.804557,0.378089,37.041888,12.851550,284.794894,0.477106,22.160354
2,Ireland,13.631311,307.503158,0.425684,22.558590,17.298813,289.965017,0.435215,16.762134


In [81]:
# Rename columns
com.columns = [
    'Channel',
    '2019',
    '2019 ',
    '2019  ',
    '2019   ',
    '2020',
    '2020 ',
    '2020  ',
    '2020   '
]

com.head()

,Channel,2019,2019,2019,2019,2020,2020,2020,2020
0,Grocery,42.178443,1504.359445,1.713361,35.666547,63.412926,1535.012311,1.744883,24.206615
1,Impulse,6.365889,235.804557,0.378089,37.041888,12.851550,284.794894,0.477106,22.160354
2,Ireland,13.631311,307.503158,0.425684,22.558590,17.298813,289.965017,0.435215,16.762134


In [82]:
alt.Chart(com).transform_fold(
    ['2019   ', '2020   '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Profit index from an Intervention'),
    color='key:N',
    column='Channel:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average Profit index from an Intervention per Channel',
    height=300,
    width=125
)

alt.Chart(...)

In [83]:
alt.Chart(com).transform_fold(
    ['2019', '2020'],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Number of Interventions'),
    color='key:N',
    column='Channel:N',
    strokeWidth=alt.value(3)
).properties(
    title = ' Average Number of Total Interventions per Channel',
    height=300,
    width=125
)

alt.Chart(...)

In [84]:
alt.Chart(com).transform_fold(
    ['2019 ', '2020 '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average revenue'),
    color='key:N',
    column='Channel:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average revenue per Channel ',
    height=300,
    width=125
)

alt.Chart(...)

In [85]:
df_2020[df_2020.Channel == 'Grocery'].Brand.value_counts()

Brand
Pralines     791
Thorntons    738
Kinder       717
Nutella      641
Bueno        531
Tic Tac      481
Name: count, dtype: int64

In [86]:
df_2020[df_2020.Channel == 'Grocery'].Group.value_counts()

Group
Tesco         1131
Asda          1067
Sainsburys     908
Morrisons      757
Booths          29
WH Smith         7
Name: count, dtype: int64

In [87]:
df_2020[df_2020.Channel == 'Impulse'].Brand.value_counts()

Brand
Pralines     260
Kinder       238
Nutella      213
Tic Tac      189
Bueno        173
Thorntons    153
Name: count, dtype: int64

In [88]:
df_2020[df_2020.Channel == 'Impulse'].Group.value_counts()

Group
Bestway         230
Dhamecha        207
Batleys         153
Retail Clubs    140
Symbols         115
Booker          106
Independent     106
Landmark         82
Parfetts         43
Todays           23
Hancocks         11
Nisa Today       10
Name: count, dtype: int64

In [89]:
df_2020[df_2020.Channel == 'Ireland'].Brand.value_counts()

Brand
Pralines     602
Kinder       597
Nutella      503
Bueno        497
Tic Tac      440
Thorntons    393
Name: count, dtype: int64

In [90]:
df_2020[df_2020.Channel == 'Ireland'].Group.value_counts()

Group
Dunnes        662
Tesco         626
BWG           440
Super Valu    415
Hendersons    286
Asda          268
Centra        193
Sainsburys    142
Name: count, dtype: int64

## Group

In [91]:
# Preview
df_2019.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali,Day
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9,1
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4,1
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7,1
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0,1
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0,1


In [92]:
# Unique values
df_2019.Group.value_counts()

Group
Tesco           1517
Asda            1298
Sainsburys      1074
Morrisons        892
Dunnes           671
Super Valu       618
Bestway          291
Booker           208
Batleys          201
Dhamecha         201
Spar             105
Landmark         104
Coop              97
Independent       95
Centra            88
Retail Clubs      80
Eurospar          78
Symbols           64
Todays            40
Hancocks          36
Parfetts          29
Vivo              23
Sugro             16
Blakemore          7
Name: count, dtype: int64

In [93]:
# Unique values
df_2020.Group.value_counts()

Group
Tesco           1757
Asda            1335
Sainsburys      1050
Morrisons        757
Dunnes           662
BWG              440
Super Valu       415
Hendersons       286
Bestway          230
Dhamecha         207
Centra           193
Batleys          153
Retail Clubs     140
Symbols          115
Booker           106
Independent      106
Landmark          82
Parfetts          43
Booths            29
Todays            23
Hancocks          11
Nisa Today        10
WH Smith           7
Name: count, dtype: int64

In [94]:
# Remove groups with less data
f = df_2019.Group.value_counts() > 50 

f = f[f.values==True]

# Preview
f.keys()

Index(['Tesco', 'Asda', 'Sainsburys', 'Morrisons', 'Dunnes', 'Super Valu',
       'Bestway', 'Booker', 'Batleys', 'Dhamecha', 'Spar', 'Landmark', 'Coop',
       'Independent', 'Centra', 'Retail Clubs', 'Eurospar', 'Symbols'],
      dtype='object', name='Group')

In [95]:
# Select columns
cols = [
    'Group',
    'Count of Interventions',
    'Net revenue',
    'Quintali',
]

In [96]:
# Group by column
fyw_2019 = df_2019[cols].groupby(
    'Group',
    
).mean().reset_index()

# Preview
fyw_2019.head()

,Group,Count of Interventions,Net revenue,Quintali
0,Asda,40.201849,1247.718310,1.472926
1,Batleys,5.611940,85.739496,0.144538
2,Bestway,8.797251,309.413333,0.579556
3,Blakemore,3.428571,0.000000,0.000000
4,Booker,6.110577,124.208000,0.232000


In [97]:
# Calculating profit index
fyw_2019['Profit index'] = fyw_2019['Net revenue'] / fyw_2019['Count of Interventions']

# Preview
fyw_2019.head()

,Group,Count of Interventions,Net revenue,Quintali,Profit index
0,Asda,40.201849,1247.718310,1.472926,31.036341
1,Batleys,5.611940,85.739496,0.144538,15.278048
2,Bestway,8.797251,309.413333,0.579556,35.171594
3,Blakemore,3.428571,0.000000,0.000000,0.000000
4,Booker,6.110577,124.208000,0.232000,20.326722


In [98]:
# Group by column
fyw_2020 = df_2020[cols].groupby(
    'Group',
    
).mean().reset_index()

# Preview
fyw_2020.head()

,Group,Count of Interventions,Net revenue,Quintali
0,Asda,57.573783,1451.889888,1.708764
1,BWG,23.554545,135.034091,0.183409
2,Batleys,8.921569,105.007812,0.197656
3,Bestway,14.104348,318.786408,0.826699
4,Booker,4.924528,155.403846,0.396154


In [99]:
# Calculating profit index
fyw_2020['Profit index'] = fyw_2020['Net revenue'] / fyw_2020['Count of Interventions']

# Preview
fyw_2020.head()

,Group,Count of Interventions,Net revenue,Quintali,Profit index
0,Asda,57.573783,1451.889888,1.708764,25.217900
1,BWG,23.554545,135.034091,0.183409,5.732825
2,Batleys,8.921569,105.007812,0.197656,11.770106
3,Bestway,14.104348,318.786408,0.826699,22.601996
4,Booker,4.924528,155.403846,0.396154,31.557103


In [100]:
# Merge
com = fyw_2019.merge(fyw_2020,on='Group')

# Preview
com.head()

,Group,Count of Interventions_x,Net revenue_x,Quintali_x,Profit index_x,Count of Interventions_y,Net revenue_y,Quintali_y,Profit index_y
0,Asda,40.201849,1247.718310,1.472926,31.036341,57.573783,1451.889888,1.708764,25.217900
1,Batleys,5.611940,85.739496,0.144538,15.278048,8.921569,105.007812,0.197656,11.770106
2,Bestway,8.797251,309.413333,0.579556,35.171594,14.104348,318.786408,0.826699,22.601996
3,Booker,6.110577,124.208000,0.232000,20.326722,4.924528,155.403846,0.396154,31.557103
4,Centra,4.693182,10.436782,0.006897,2.223818,4.854922,44.891192,0.050777,9.246531


In [101]:
# Rename columns
com.columns = [
    'Group',
    '2019',
    '2019 ',
    '2019  ',
    '2019   ',
    '2020',
    '2020 ',
    '2020  ',
    '2020   '
]

com.head()

,Group,2019,2019,2019,2019,2020,2020,2020,2020
0,Asda,40.201849,1247.718310,1.472926,31.036341,57.573783,1451.889888,1.708764,25.217900
1,Batleys,5.611940,85.739496,0.144538,15.278048,8.921569,105.007812,0.197656,11.770106
2,Bestway,8.797251,309.413333,0.579556,35.171594,14.104348,318.786408,0.826699,22.601996
3,Booker,6.110577,124.208000,0.232000,20.326722,4.924528,155.403846,0.396154,31.557103
4,Centra,4.693182,10.436782,0.006897,2.223818,4.854922,44.891192,0.050777,9.246531


In [102]:
alt.Chart(com).transform_fold(
    ['2019   ', '2020   '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Profit index from an Intervention'),
    color='key:N',
    column='Group:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average Profit index from an Intervention per Group',
    height=300,
    width=35
)

alt.Chart(...)

In [103]:
alt.Chart(com).transform_fold(
    ['2019', '2020'],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Number of Interventions'),
    color='key:N',
    column='Group:N',
    strokeWidth=alt.value(3)
).properties(
    title = ' Average Number of Total Interventions per Group',
    height=300,
    width=35
)

alt.Chart(...)

In [104]:
alt.Chart(com).transform_fold(
    ['2019 ', '2020 '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average revenue'),
    color='key:N',
    column='Group:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average revenue per Group ',
    height=300,
    width=35
)

alt.Chart(...)

## Brand

In [105]:
# Preview
df_2019.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali,Day
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9,1
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4,1
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7,1
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0,1
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0,1


In [106]:
# Unique values
df_2019.Brand.value_counts()

Brand
Pralines     1648
Kinder       1543
Thorntons    1366
Nutella      1269
Bueno        1092
Tic Tac       915
Name: count, dtype: int64

In [144]:
# Unique values
df_2020.Brand.value_counts()

Brand
Pralines     1653
Kinder       1552
Nutella      1357
Thorntons    1284
Bueno        1201
Tic Tac      1110
Name: count, dtype: int64

In [107]:
# Select columns
cols = [
    'Brand',
    'Count of Interventions',
    'Net revenue',
    'Quintali'
]

In [108]:
# Group by column
fyw_2019 = df_2019[cols].groupby(
    'Brand',
    
).mean().reset_index()

# Preview
fyw_2019.head()

,Brand,Count of Interventions,Net revenue,Quintali
0,Bueno,12.248168,186.029268,0.344098
1,Kinder,26.620868,426.231716,0.450239
2,Nutella,17.573680,417.881054,1.232455
3,Pralines,41.219660,2208.846598,2.076893
4,Thorntons,40.991947,1549.968429,1.785399


In [109]:
# Calculating profit index
fyw_2019['Profit index'] = fyw_2019['Net revenue'] / fyw_2019['Count of Interventions']

# Preview
fyw_2019.head()

,Brand,Count of Interventions,Net revenue,Quintali,Profit index
0,Bueno,12.248168,186.029268,0.344098,15.188334
1,Kinder,26.620868,426.231716,0.450239,16.011187
2,Nutella,17.573680,417.881054,1.232455,23.778802
3,Pralines,41.219660,2208.846598,2.076893,53.587210
4,Thorntons,40.991947,1549.968429,1.785399,37.811535


In [110]:
# Group by column
fyw_2020 = df_2020[cols].groupby(
    'Brand',
    
).mean().reset_index()

# Preview
fyw_2020.head()

,Brand,Count of Interventions,Net revenue,Quintali
0,Bueno,21.992506,212.448793,0.412739
1,Kinder,44.829253,521.059470,0.521396
2,Nutella,31.731024,490.273739,1.487463
3,Pralines,53.414398,2534.520998,2.436945
4,Thorntons,55.468847,1004.623520,1.060695


In [111]:
# Calculating profit index
fyw_2020['Profit index'] = fyw_2020['Net revenue'] / fyw_2020['Count of Interventions']

# Preview
fyw_2020.head()

,Brand,Count of Interventions,Net revenue,Quintali,Profit index
0,Bueno,21.992506,212.448793,0.412739,9.660054
1,Kinder,44.829253,521.059470,0.521396,11.623202
2,Nutella,31.731024,490.273739,1.487463,15.450927
3,Pralines,53.414398,2534.520998,2.436945,47.450146
4,Thorntons,55.468847,1004.623520,1.060695,18.111491


In [112]:
# Merge
com = fyw_2019.merge(fyw_2020,on='Brand')

# Preview
com.head()

,Brand,Count of Interventions_x,Net revenue_x,Quintali_x,Profit index_x,Count of Interventions_y,Net revenue_y,Quintali_y,Profit index_y
0,Bueno,12.248168,186.029268,0.344098,15.188334,21.992506,212.448793,0.412739,9.660054
1,Kinder,26.620868,426.231716,0.450239,16.011187,44.829253,521.059470,0.521396,11.623202
2,Nutella,17.573680,417.881054,1.232455,23.778802,31.731024,490.273739,1.487463,15.450927
3,Pralines,41.219660,2208.846598,2.076893,53.587210,53.414398,2534.520998,2.436945,47.450146
4,Thorntons,40.991947,1549.968429,1.785399,37.811535,55.468847,1004.623520,1.060695,18.111491


In [113]:
# Rename columns
com.columns = [
    'Brand',
    '2019',
    '2019 ',
    '2019  ',
    '2019   ',
    '2020',
    '2020 ',
    '2020  ',
    '2020   '
]

com.head()

,Brand,2019,2019,2019,2019,2020,2020,2020,2020
0,Bueno,12.248168,186.029268,0.344098,15.188334,21.992506,212.448793,0.412739,9.660054
1,Kinder,26.620868,426.231716,0.450239,16.011187,44.829253,521.059470,0.521396,11.623202
2,Nutella,17.573680,417.881054,1.232455,23.778802,31.731024,490.273739,1.487463,15.450927
3,Pralines,41.219660,2208.846598,2.076893,53.587210,53.414398,2534.520998,2.436945,47.450146
4,Thorntons,40.991947,1549.968429,1.785399,37.811535,55.468847,1004.623520,1.060695,18.111491


In [114]:
alt.Chart(com).transform_fold(
    ['2019   ', '2020   '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Profit index from an Intervention'),
    color='key:N',
    column='Brand:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average Profit index from an Intervention per Brand',
    height=300,
    width=100
)

alt.Chart(...)

In [115]:
alt.Chart(com).transform_fold(
    ['2019', '2020'],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Number of Interventions'),
    color='key:N',
    column='Brand:N',
    strokeWidth=alt.value(3)
).properties(
    title = ' Average Number of Total Interventions per Brand',
    height=300,
    width=100
)

alt.Chart(...)

In [116]:
alt.Chart(com).transform_fold(
    ['2019 ', '2020 '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average revenue'),
    color='key:N',
    column='Brand:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average revenue per Brand ',
    height=300,
    width=100
)

alt.Chart(...)

## Top Intervention

In [117]:
# Preview
df_2019.head()

,FinancialYear,FYWeek,WeekComm,Date,Channel,Group,Brand,Top Intervention,Count of Interventions,Net revenue,Quintali,Day
0,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Amend Stock Level,26,1586.0,2.9,1
1,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Local Display,5,241.0,0.4,1
2,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Merchandising,25,424.0,0.7,1
3,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,National Display,6,428.0,1.0,1
4,20182019,13,2018-11-25,2018-12-01,Grocery,Asda,Bueno,Place Order,1,13.0,0.0,1


In [118]:
# Unique values
df_2019['Top Intervention'].value_counts()

Top Intervention
Replenishment                  1449
Merchandising                  1433
Local Display                  1057
Place Order                     840
Site SEL / POS                  771
Amend Stock Level               534
Permanent Additional Facing     503
Sited Clipstrips                345
National Display                276
No Action                       268
No Intervention                  76
LVA - Merchandised               69
Site SEL/POS                     63
Distribution Gain                55
HVA - Local Display              43
HVA - Sited POS                  23
LVA - Replenishment              15
HVA - Distribution Gain          11
LVA - Sited SEL                   1
HVA - Placed Order                1
Name: count, dtype: int64

In [154]:
# Unique values
df_2020['Top Intervention'].value_counts()

Top Intervention
Merchandising                  1778
Replenishment                  1423
Local Display                  1159
Site SEL / POS                  830
Place Order                     644
Fix Book stock error            571
Adjust shelf capacity           374
National Display                352
Sited Clipstrips                286
Permanent Additional Facing     258
No Intervention                 149
Merchandised                    133
No Action                        72
Distribution Gain                66
Placed Order                     26
Site SEL/ POS                    24
Amend Stock Level                 6
Sited Clip Strips                 3
Sited POS                         3
Name: count, dtype: int64

In [119]:
# Remove groups with less data
f = df_2019['Top Intervention'].value_counts() > 50 

f = f[f.values==True]

# Preview
f.keys()

Index(['Replenishment', 'Merchandising', 'Local Display', 'Place Order',
       'Site SEL / POS', 'Amend Stock Level', 'Permanent Additional Facing',
       'Sited Clipstrips', 'National Display', 'No Action', 'No Intervention',
       'LVA - Merchandised', 'Site SEL/POS', 'Distribution Gain'],
      dtype='object', name='Top Intervention')

In [120]:
# Select columns
cols = [
    'Top Intervention',
    'Count of Interventions',
    'Net revenue',
    'Quintali'
]

In [121]:
# Group by column
fyw_2019 = df_2019[cols].groupby(
    'Top Intervention',
    
).mean().reset_index()

# Preview
fyw_2019.head()

,Top Intervention,Count of Interventions,Net revenue,Quintali
0,Amend Stock Level,30.054307,1613.032015,2.233333
1,Distribution Gain,1.200000,18.038462,0.019231
2,HVA - Distribution Gain,1.636364,136.727273,0.100000
3,HVA - Local Display,12.581395,1155.093023,1.004651
4,HVA - Placed Order,1.000000,0.000000,0.000000


In [122]:
# Calculating profit index
fyw_2019['Profit index'] = fyw_2019['Net revenue'] / fyw_2019['Count of Interventions']

# Preview
fyw_2019.head()

,Top Intervention,Count of Interventions,Net revenue,Quintali,Profit index
0,Amend Stock Level,30.054307,1613.032015,2.233333,53.670577
1,Distribution Gain,1.200000,18.038462,0.019231,15.032051
2,HVA - Distribution Gain,1.636364,136.727273,0.100000,83.555556
3,HVA - Local Display,12.581395,1155.093023,1.004651,91.809612
4,HVA - Placed Order,1.000000,0.000000,0.000000,0.000000


In [123]:
# Group by column
fyw_2020 = df_2020[cols].groupby(
    'Top Intervention',
    
).mean().reset_index()

# Preview
fyw_2020.head()

,Top Intervention,Count of Interventions,Net revenue,Quintali
0,Adjust shelf capacity,8.810160,618.858289,0.767647
1,Amend Stock Level,2.833333,9.000000,0.000000
2,Distribution Gain,1.909091,27.454545,0.019697
3,Fix Book stock error,23.057793,833.075306,1.219440
4,Local Display,31.012079,1315.151986,1.357427


In [124]:
# Calculating profit index
fyw_2020['Profit index'] = fyw_2020['Net revenue'] / fyw_2020['Count of Interventions']

# Preview
fyw_2020.head()

,Top Intervention,Count of Interventions,Net revenue,Quintali,Profit index
0,Adjust shelf capacity,8.810160,618.858289,0.767647,70.243703
1,Amend Stock Level,2.833333,9.000000,0.000000,3.176471
2,Distribution Gain,1.909091,27.454545,0.019697,14.380952
3,Fix Book stock error,23.057793,833.075306,1.219440,36.129880
4,Local Display,31.012079,1315.151986,1.357427,42.407733


In [125]:
# Merge
com = fyw_2019.merge(fyw_2020,on='Top Intervention')

# Preview
com.head()

,Top Intervention,Count of Interventions_x,Net revenue_x,Quintali_x,Profit index_x,Count of Interventions_y,Net revenue_y,Quintali_y,Profit index_y
0,Amend Stock Level,30.054307,1613.032015,2.233333,53.670577,2.833333,9.000000,0.000000,3.176471
1,Distribution Gain,1.200000,18.038462,0.019231,15.032051,1.909091,27.454545,0.019697,14.380952
2,Local Display,44.212867,2277.659091,2.404348,51.515752,31.012079,1315.151986,1.357427,42.407733
3,Merchandising,26.676902,454.757196,0.651070,17.046852,61.237908,772.794151,1.137683,12.619539
4,National Display,10.083333,532.178832,0.528832,52.778066,15.275568,1098.818182,1.196875,71.933048


In [126]:
# Rename columns
com.columns = [
    'Top Intervention',
    '2019',
    '2019 ',
    '2019  ',
    '2019   ',
    '2020',
    '2020 ',
    '2020  ',
    '2020   '
]

com.head()

,Top Intervention,2019,2019,2019,2019,2020,2020,2020,2020
0,Amend Stock Level,30.054307,1613.032015,2.233333,53.670577,2.833333,9.000000,0.000000,3.176471
1,Distribution Gain,1.200000,18.038462,0.019231,15.032051,1.909091,27.454545,0.019697,14.380952
2,Local Display,44.212867,2277.659091,2.404348,51.515752,31.012079,1315.151986,1.357427,42.407733
3,Merchandising,26.676902,454.757196,0.651070,17.046852,61.237908,772.794151,1.137683,12.619539
4,National Display,10.083333,532.178832,0.528832,52.778066,15.275568,1098.818182,1.196875,71.933048


In [127]:
alt.Chart(com[com['Top Intervention'].isin(f.keys())]).transform_fold(
    ['2019   ', '2020   '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Profit index from an Intervention'),
    color='key:N',
    column='Top Intervention:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average PI from an Intervention per Top Intervention',
    height=300,
    width=80
)

alt.Chart(...)

In [128]:
alt.Chart(com).transform_fold(
    ['2019', '2020'],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average Number of Interventions'),
    color='key:N',
    column='Top Intervention:N',
    strokeWidth=alt.value(3)
).properties(
    title = ' Average Number of Total Interventions per Top Intervention',
    height=300,
    width=80
)

alt.Chart(...)

In [129]:
alt.Chart(com).transform_fold(
    ['2019 ', '2020 '],
).mark_bar().encode(
    x=alt.X('key:O',title = 'Year'),
    y=alt.Y('value:Q', title='Average revenue'),
    color='key:N',
    column='Top Intervention:N',
    strokeWidth=alt.value(3)
).properties(
    title = 'Average revenue per Top Intervention ',
    height=300,
    width=80
)

alt.Chart(...)